In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


In [3]:
# Load the dataset
data = pd.read_csv("C:/Users/LENOVO/Downloads/bank_transactions_data_2.csv")

# Display basic info
print(data.info())
print(data.describe())

# Count missing values
missing_values = data.isnull().sum()
print("Missing values:\n", missing_values)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2512 entries, 0 to 2511
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   TransactionID            2512 non-null   object 
 1   AccountID                2512 non-null   object 
 2   TransactionAmount        2512 non-null   float64
 3   TransactionDate          2512 non-null   object 
 4   TransactionType          2512 non-null   object 
 5   Location                 2512 non-null   object 
 6   DeviceID                 2512 non-null   object 
 7   IP Address               2512 non-null   object 
 8   MerchantID               2512 non-null   object 
 9   Channel                  2512 non-null   object 
 10  CustomerAge              2512 non-null   int64  
 11  CustomerOccupation       2512 non-null   object 
 12  TransactionDuration      2512 non-null   int64  
 13  LoginAttempts            2512 non-null   int64  
 14  AccountBalance          

There are no missing values. It was checked to see if there are missing values. But it was mentioned that there are no missing values. Eventhough there are no missing values , I'm going to set the median of the each column to the missing values if they are found.

In [4]:
# Replace missing values with the median of respective columns
data.fillna(data.median(numeric_only=True), inplace=True)

# Verify no missing values remain
print("Missing values after replacement:\n", data.isnull().sum())


Missing values after replacement:
 TransactionID              0
AccountID                  0
TransactionAmount          0
TransactionDate            0
TransactionType            0
Location                   0
DeviceID                   0
IP Address                 0
MerchantID                 0
Channel                    0
CustomerAge                0
CustomerOccupation         0
TransactionDuration        0
LoginAttempts              0
AccountBalance             0
PreviousTransactionDate    0
dtype: int64


Clearly shown that there are no missing values now.

In [8]:
print(data.columns)




Index(['TransactionAmount', 'CustomerAge', 'TransactionDuration',
       'LoginAttempts', 'AccountBalance', 'TransactionID_TX000002',
       'TransactionID_TX000003', 'TransactionID_TX000004',
       'TransactionID_TX000005', 'TransactionID_TX000006',
       ...
       'PreviousTransactionDate_2024-11-04 08:12:14',
       'PreviousTransactionDate_2024-11-04 08:12:15',
       'PreviousTransactionDate_2024-11-04 08:12:16',
       'PreviousTransactionDate_2024-11-04 08:12:17',
       'PreviousTransactionDate_2024-11-04 08:12:18',
       'PreviousTransactionDate_2024-11-04 08:12:19',
       'PreviousTransactionDate_2024-11-04 08:12:20',
       'PreviousTransactionDate_2024-11-04 08:12:21',
       'PreviousTransactionDate_2024-11-04 08:12:22',
       'PreviousTransactionDate_2024-11-04 08:12:23'],
      dtype='object', length=7298)


In [13]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

# Define relevant columns for logistic regression
features = ['TransactionAmount', 'TransactionDuration', 'LoginAttempts', 'AccountBalance','CustomerAge','TimeSinceLastTransaction']
target = 'Fraud'

# Prepare the dataset
X = data[features]
y = data[target]


# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Train Logistic Regression model
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

# Predict fraud on the test set
y_pred = log_reg.predict(X_test)




# Evaluate model performance
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Non-Fraud', 'Fraud'], yticklabels=['Non-Fraud', 'Fraud'])
plt.title('Confusion Matrix', fontsize=16)
plt.xlabel('Predicted', fontsize=14)
plt.ylabel('Actual', fontsize=14)
plt.show()

# Add predictions to the dataset
data['LogReg_Fraud'] = log_reg.predict(X_scaled)
data['Fraud'] |= data['LogReg_Fraud']



# Visualize fraud vs. non-fraud transactions
plt.figure(figsize=(12, 8))
sns.scatterplot(
    x=data['TransactionAmount'],
    y=data['AccountBalance'],
    hue=data['LogReg_Fraud'],
    palette={1: 'red', 0: 'blue'},
    alpha=0.7
)
plt.title('Logistic Regression Fraud Detection', fontsize=16)
plt.xlabel('Transaction Amount', fontsize=14)
plt.ylabel('Account Balance', fontsize=14)
plt.legend(title='Fraud', labels=['Non-Fraud', 'Fraud'], fontsize=12)
plt.grid(True)
plt.show()

# Save fraudulent transactions detected by Logistic Regression
log_reg_fraud_output_path = '/kaggle/working/log_reg_fraud_transactions.csv'
log_reg_fraud_points = data[data['LogReg_Fraud'] == 1]
log_reg_fraud_points.to_csv(log_reg_fraud_output_path, index=False)

# Summary
total_log_reg_fraud_points = log_reg_fraud_points.shape[0]
print(f"Total Fraudulent Transactions Detected by Logistic Regression: {total_log_reg_fraud_points}")
print(f"Fraudulent transactions saved to: {log_reg_fraud_output_path}")
print(log_reg_fraud_points)

KeyError: "['TimeSinceLastTransaction'] not in index"

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
# Load the dataset
file_path = "C:/Users/LENOVO/Downloads/bank_transactions_data_2.csv"
data = pd.read_csv(file_path)

# Display initial information about the dataset
print(data.head())      # Preview first few rows
print(data.info())      # Structure and types of data
print(data.describe())  # Summary statistics for numerical columns

  TransactionID AccountID  TransactionAmount      TransactionDate  \
0      TX000001   AC00128              14.09  2023-04-11 16:29:14   
1      TX000002   AC00455             376.24  2023-06-27 16:44:19   
2      TX000003   AC00019             126.29  2023-07-10 18:16:08   
3      TX000004   AC00070             184.50  2023-05-05 16:32:11   
4      TX000005   AC00411              13.45  2023-10-16 17:51:24   

  TransactionType   Location DeviceID      IP Address MerchantID Channel  \
0           Debit  San Diego  D000380  162.198.218.92       M015     ATM   
1           Debit    Houston  D000051     13.149.61.4       M052     ATM   
2           Debit       Mesa  D000235  215.97.143.157       M009  Online   
3           Debit    Raleigh  D000187  200.13.225.150       M002  Online   
4          Credit    Atlanta  D000308    65.164.3.100       M091  Online   

   CustomerAge CustomerOccupation  TransactionDuration  LoginAttempts  \
0           70             Doctor                   81 

In [16]:

# Check for missing values
print("Missing values:\n", data.isnull().sum())

# Fill or drop missing values (example: fill missing numeric columns with median)
numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].median())

# Handle categorical data (example: fill missing with mode)
categorical_columns = data.select_dtypes(include=['object']).columns
for col in categorical_columns:
    data[col] = data[col].fillna(data[col].mode()[0])

# Convert date columns to datetime format (if applicable)
if 'date' in data.columns:
    data['date'] = pd.to_datetime(data['date'])

# Select only numeric columns
numeric_data = data.select_dtypes(include=['float64', 'int64'])

# Drop columns with too many missing values (optional)
numeric_data = numeric_data.dropna(axis=1, thresh=len(numeric_data) * 0.5)  # Keep columns with >50% non-NaN values

# Fill remaining missing values with median (optional)
numeric_data = numeric_data.fillna(numeric_data.median())


Missing values:
 TransactionID              0
AccountID                  0
TransactionAmount          0
TransactionDate            0
TransactionType            0
Location                   0
DeviceID                   0
IP Address                 0
MerchantID                 0
Channel                    0
CustomerAge                0
CustomerOccupation         0
TransactionDuration        0
LoginAttempts              0
AccountBalance             0
PreviousTransactionDate    0
dtype: int64


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

# Define relevant columns for logistic regression
features = ['TransactionAmount', 'TransactionDuration', 'LoginAttempts', 'AccountBalance','CustomerAge','TimeSinceLastTransaction']
target = 'Fraud'

# Prepare the dataset
X = data[features]
y = data[target]


# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Train Logistic Regression model
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

# Predict fraud on the test set
y_pred = log_reg.predict(X_test)




# Evaluate model performance
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Non-Fraud', 'Fraud'], yticklabels=['Non-Fraud', 'Fraud'])
plt.title('Confusion Matrix', fontsize=16)
plt.xlabel('Predicted', fontsize=14)
plt.ylabel('Actual', fontsize=14)
plt.show()

# Add predictions to the dataset
data['LogReg_Fraud'] = log_reg.predict(X_scaled)
data['Fraud'] |= data['LogReg_Fraud']



# Visualize fraud vs. non-fraud transactions
plt.figure(figsize=(12, 8))
sns.scatterplot(
    x=data['TransactionAmount'],
    y=data['AccountBalance'],
    hue=data['LogReg_Fraud'],
    palette={1: 'red', 0: 'blue'},
    alpha=0.7
)
plt.title('Logistic Regression Fraud Detection', fontsize=16)
plt.xlabel('Transaction Amount', fontsize=14)
plt.ylabel('Account Balance', fontsize=14)
plt.legend(title='Fraud', labels=['Non-Fraud', 'Fraud'], fontsize=12)
plt.grid(True)
plt.show()

# Save fraudulent transactions detected by Logistic Regression
log_reg_fraud_output_path = '/kaggle/working/log_reg_fraud_transactions.csv'
log_reg_fraud_points = data[data['LogReg_Fraud'] == 1]
log_reg_fraud_points.to_csv(log_reg_fraud_output_path, index=False)

# Summary
total_log_reg_fraud_points = log_reg_fraud_points.shape[0]
print(f"Total Fraudulent Transactions Detected by Logistic Regression: {total_log_reg_fraud_points}")
print(f"Fraudulent transactions saved to: {log_reg_fraud_output_path}")
print(log_reg_fraud_points)

KeyError: "['TimeSinceLastTransaction'] not in index"

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

# Define relevant columns for logistic regression
features = ['TransactionAmount', 'TransactionDuration', 'LoginAttempts', 'AccountBalance', 'CustomerAge', 'TimeSinceLastTransaction']
target = 'Fraud'

# Prepare the dataset
X = data[features]
y = data[target]

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Train Logistic Regression model
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

# Predict fraud on the test set
y_pred = log_reg.predict(X_test)

# Evaluate model performance
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Non-Fraud', 'Fraud'], yticklabels=['Non-Fraud', 'Fraud'])
plt.title('Confusion Matrix', fontsize=16)
plt.xlabel('Predicted', fontsize=14)
plt.ylabel('Actual', fontsize=14)
plt.show()

# Add predictions to the dataset
data['LogReg_Fraud'] = log_reg.predict(X_scaled)

# If you want to set 'Fraud' column to True wherever the model predicts fraud
data['Fraud'] = data['Fraud'].fillna(0)  # Assuming Fraud column exists and has NaN values
data['Fraud'] |= data['LogReg_Fraud']

# Visualize fraud vs. non-fraud transactions
plt.figure(figsize=(12, 8))
sns.scatterplot(
    x=data['TransactionAmount'],
    y=data['AccountBalance'],
    hue=data['LogReg_Fraud'],
    palette={1: 'red', 0: 'blue'},
    alpha=0.7
)
plt.title('Logistic Regression Fraud Detection', fontsize=16)
plt.xlabel('Transaction Amount', fontsize=14)
plt.ylabel('Account Balance', fontsize=14)
plt.legend(title='Fraud', labels=['Non-Fraud', 'Fraud'], fontsize=12)
plt.grid(True)
plt.show()

# Save fraudulent transactions detected by Logistic Regression
log_reg_fraud_output_path = '/kaggle/working/log_reg_fraud_transactions.csv'
log_reg_fraud_points = data[data['LogReg_Fraud'] == 1]
log_reg_fraud_points.to_csv(log_reg_fraud_output_path, index=False)

# Summary
total_log_reg_fraud_points = log_reg_fraud_points.shape[0]
print(f"Total Fraudulent Transactions Detected by Logistic Regression: {total_log_reg_fraud_points}")
print(f"Fraudulent transactions saved to: {log_reg_fraud_output_path}")
print(log_reg_fraud_points.head())


KeyError: "['TimeSinceLastTransaction'] not in index"

In [19]:
print(data.columns)


Index(['TransactionID', 'AccountID', 'TransactionAmount', 'TransactionDate',
       'TransactionType', 'Location', 'DeviceID', 'IP Address', 'MerchantID',
       'Channel', 'CustomerAge', 'CustomerOccupation', 'TransactionDuration',
       'LoginAttempts', 'AccountBalance', 'PreviousTransactionDate'],
      dtype='object')


In [20]:
# Check the columns in your dataset
print(data.columns)

# Update the feature list if necessary
features = ['TransactionAmount', 'TransactionDuration', 'LoginAttempts', 'AccountBalance', 'CustomerAge', 'TimeSinceLastTransaction']

# Make sure these columns exist in your DataFrame, or update them accordingly
X = data[features]  # This should now work if the columns are present


Index(['TransactionID', 'AccountID', 'TransactionAmount', 'TransactionDate',
       'TransactionType', 'Location', 'DeviceID', 'IP Address', 'MerchantID',
       'Channel', 'CustomerAge', 'CustomerOccupation', 'TransactionDuration',
       'LoginAttempts', 'AccountBalance', 'PreviousTransactionDate'],
      dtype='object')


KeyError: "['TimeSinceLastTransaction'] not in index"

In [21]:
features = ['TransactionAmount', 'TransactionDuration', 'LoginAttempts', 'AccountBalance', 'CustomerAge']


In [22]:
# Convert date columns to datetime
data['TransactionDate'] = pd.to_datetime(data['TransactionDate'])
data['PreviousTransactionDate'] = pd.to_datetime(data['PreviousTransactionDate'])

# Calculate time difference in seconds (or any other time unit you prefer)
data['TimeSinceLastTransaction'] = (data['TransactionDate'] - data['PreviousTransactionDate']).dt.total_seconds()

# Now you can include 'TimeSinceLastTransaction' in your features list
features = ['TransactionAmount', 'TransactionDuration', 'LoginAttempts', 'AccountBalance', 'CustomerAge', 'TimeSinceLastTransaction']


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Prepare the dataset
features = ['TransactionAmount', 'TransactionDuration', 'LoginAttempts', 'AccountBalance', 'CustomerAge', 'TimeSinceLastTransaction']  # Update this as needed
target = 'TransactionType'

X = data[features]
y = data[target].map({'Non-Fraud': 0, 'Fraud': 1})  # Encode target variable

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Train Logistic Regression model
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

# Predict fraud on the test set
y_pred = log_reg.predict(X_test)

# Evaluate model performance
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Non-Fraud', 'Fraud'], yticklabels=['Non-Fraud', 'Fraud'])
plt.title('Confusion Matrix', fontsize=16)
plt.xlabel('Predicted', fontsize=14)
plt.ylabel('Actual', fontsize=14)
plt.show()

# Add predictions to the dataset
data['LogReg_Fraud'] = log_reg.predict(X_scaled)
data['Fraud'] |= data['LogReg_Fraud']

# Visualize fraud vs. non-fraud transactions
plt.figure(figsize=(12, 8))
sns.scatterplot(
    x=data['TransactionAmount'],
    y=data['AccountBalance'],
    hue=data['LogReg_Fraud'],
    palette={1: 'red', 0: 'blue'},
    alpha=0.7
)
plt.title('Logistic Regression Fraud Detection', fontsize=16)
plt.xlabel('Transaction Amount', fontsize=14)
plt.ylabel('Account Balance', fontsize=14)
plt.legend(title='Fraud', labels=['Non-Fraud', 'Fraud'], fontsize=12)
plt.grid(True)
plt.show()

# Save fraudulent transactions detected by Logistic Regression
log_reg_fraud_output_path = '/kaggle/working/log_reg_fraud_transactions.csv'
log_reg_fraud_points = data[data['LogReg_Fraud'] == 1]
log_reg_fraud_points.to_csv(log_reg_fraud_output_path, index=False)

# Summary
total_log_reg_fraud_points = log_reg_fraud_points.shape[0]
print(f"Total Fraudulent Transactions Detected by Logistic Regression: {total_log_reg_fraud_points}")
print(f"Fraudulent transactions saved to: {log_reg_fraud_output_path}")
print(log_reg_fraud_points)


ValueError: Input y contains NaN.

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Drop irrelevant columns
columns_to_drop = ['TransactionID', 'PreviousTransactionDate', 'TransactionDate', 'IP Address', 'DeviceID']
data = data.drop(columns=columns_to_drop, axis=1)

# Encode the target variable
data['TransactionType'] = data['TransactionType'].map({'Non-Fraud': 0, 'Fraud': 1})

# One-hot encode categorical columns
categorical_columns = ['Location', 'MerchantID', 'Channel', 'CustomerOccupation']
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# Fill missing values in features
data = data.fillna(data.mean())  # Fill missing numeric features with the mean

# Ensure 'Fraud' is numeric
y = data['TransactionType'].astype(int)

# Define features for logistic regression (use only numeric columns after encoding)
features = ['TransactionAmount', 'TransactionDuration', 'LoginAttempts', 'AccountBalance', 'CustomerAge']
features.extend([col for col in data.columns if col not in ['TransactionType']])  # Add one-hot encoded columns

# Prepare the dataset for modeling
X = data[features]

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Train Logistic Regression model
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

# Predict fraud on the test set
y_pred = log_reg.predict(X_test)

# Evaluate model performance
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Non-Fraud', 'Fraud'], yticklabels=['Non-Fraud', 'Fraud'])
plt.title('Confusion Matrix', fontsize=16)
plt.xlabel('Predicted', fontsize=14)
plt.ylabel('Actual', fontsize=14)
plt.show()

# Add predictions to the dataset
data['LogReg_Fraud'] = log_reg.predict(X_scaled)

# Visualize fraud vs. non-fraud transactions
plt.figure(figsize=(12, 8))
sns.scatterplot(
    x=data['TransactionAmount'],
    y=data['AccountBalance'],
    hue=data['LogReg_Fraud'],
    palette={1: 'red', 0: 'blue'},
    alpha=0.7
)
plt.title('Logistic Regression Fraud Detection', fontsize=16)
plt.xlabel('Transaction Amount', fontsize=14)
plt.ylabel('Account Balance', fontsize=14)
plt.legend(title='Fraud', labels=['Non-Fraud', 'Fraud'], fontsize=12)
plt.grid(True)
plt.show()

# Save fraudulent transactions detected by Logistic Regression
log_reg_fraud_output_path = 'log_reg_fraud_transactions.csv'
log_reg_fraud_points = data[data['LogReg_Fraud'] == 1]
log_reg_fraud_points.to_csv(log_reg_fraud_output_path, index=False)

# Summary
total_log_reg_fraud_points = log_reg_fraud_points.shape[0]
print(f"Total Fraudulent Transactions Detected by Logistic Regression: {total_log_reg_fraud_points}")
print(f"Fraudulent transactions saved to: {log_reg_fraud_output_path}")
print(log_reg_fraud_points)


TypeError: Could not convert ['AC00128AC00455AC00019AC00070AC00411AC00393AC00199AC00069AC00135AC00385AC00150AC00459AC00392AC00264AC00085AC00270AC00317AC00359AC00242AC00285AC00002AC00014AC00095AC00453AC00241AC00041AC00441AC00057AC00390AC00313AC00367AC00291AC00060AC00359AC00365AC00267AC00404AC00202AC00478AC00405AC00421AC00480AC00120AC00117AC00011AC00401AC00282AC00439AC00296AC00471AC00159AC00482AC00115AC00055AC00419AC00374AC00271AC00456AC00143AC00427AC00438AC00002AC00425AC00325AC00434AC00242AC00319AC00460AC00461AC00395AC00217AC00053AC00282AC00110AC00265AC00239AC00007AC00022AC00464AC00014AC00406AC00445AC00236AC00220AC00140AC00098AC00303AC00373AC00438AC00437AC00106AC00310AC00149AC00373AC00425AC00010AC00348AC00011AC00453AC00178AC00265AC00018AC00075AC00464AC00170AC00235AC00013AC00397AC00423AC00154AC00068AC00318AC00396AC00121AC00336AC00468AC00282AC00498AC00085AC00092AC00295AC00358AC00311AC00066AC00340AC00284AC00208AC00176AC00053AC00320AC00382AC00384AC00203AC00498AC00349AC00177AC00284AC00272AC0

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Drop irrelevant columns
columns_to_drop = ['TransactionID', 'PreviousTransactionDate', 'TransactionDate', 'IP Address', 'DeviceID']
data = data.drop(columns=columns_to_drop, axis=1)

# Encode the target variable
# Ensure that 'TransactionType' is properly mapped and handle missing values before conversion
data['TransactionType'] = data['TransactionType'].map({'Non-Fraud': 0, 'Fraud': 1})

# Check if any missing values in 'TransactionType'
if data['TransactionType'].isnull().any():
    print("Missing values found in 'TransactionType', filling with 0 (Non-Fraud).")
    data['TransactionType'].fillna(0, inplace=True)  # Fill missing values with 0 for non-fraud

# One-hot encode categorical columns
categorical_columns = ['Location', 'MerchantID', 'Channel', 'CustomerOccupation']
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# Fill missing values in features
data = data.fillna(data.mean())  # Fill missing numeric features with the mean

# Ensure 'Fraud' column is numeric and free of issues
y = data['TransactionType'].astype(int)

# Define features for logistic regression (use only numeric columns after encoding)
features = ['TransactionAmount', 'TransactionDuration', 'LoginAttempts', 'AccountBalance', 'CustomerAge']
features.extend([col for col in data.columns if col not in ['TransactionType']])  # Add one-hot encoded columns

# Prepare the dataset for modeling
X = data[features]

# Check if X contains any non-numeric data
if X.select_dtypes(include='object').shape[1] > 0:
    print("Non-numeric columns detected in features. Converting to numeric.")
    X = X.apply(pd.to_numeric, errors='coerce')  # Convert non-numeric values to NaN

# Handle any missing values that may appear after conversion
X.fillna(X.mean(), inplace=True)

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Train Logistic Regression model
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

# Predict fraud on the test set
y_pred = log_reg.predict(X_test)

# Evaluate model performance
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Non-Fraud', 'Fraud'], yticklabels=['Non-Fraud', 'Fraud'])
plt.title('Confusion Matrix', fontsize=16)
plt.xlabel('Predicted', fontsize=14)
plt.ylabel('Actual', fontsize=14)
plt.show()

# Add predictions to the dataset
data['LogReg_Fraud'] = log_reg.predict(X_scaled)

# Visualize fraud vs. non-fraud transactions
plt.figure(figsize=(12, 8))
sns.scatterplot(
    x=data['TransactionAmount'],
    y=data['AccountBalance'],
    hue=data['LogReg_Fraud'],
    palette={1: 'red', 0: 'blue'},
    alpha=0.7
)
plt.title('Logistic Regression Fraud Detection', fontsize=16)
plt.xlabel('Transaction Amount', fontsize=14)
plt.ylabel('Account Balance', fontsize=14)
plt.legend(title='Fraud', labels=['Non-Fraud', 'Fraud'], fontsize=12)
plt.grid(True)
plt.show()

# Save fraudulent transactions detected by Logistic Regression
log_reg_fraud_output_path = 'log_reg_fraud_transactions.csv'
log_reg_fraud_points = data[data['LogReg_Fraud'] == 1]
log_reg_fraud_points.to_csv(log_reg_fraud_output_path, index=False)

# Summary
total_log_reg_fraud_points = log_reg_fraud_points.shape[0]
print(f"Total Fraudulent Transactions Detected by Logistic Regression: {total_log_reg_fraud_points}")
print(f"Fraudulent transactions saved to: {log_reg_fraud_output_path}")
print(log_reg_fraud_points)


KeyError: "['TransactionID', 'PreviousTransactionDate', 'TransactionDate', 'IP Address', 'DeviceID'] not found in axis"

In [27]:
data.columns = data.columns.str.strip()  # Remove any leading/trailing spaces


In [28]:
# Drop irrelevant columns if they exist
columns_to_drop = ['TransactionID', 'PreviousTransactionDate', 'TransactionDate', 'IP Address', 'DeviceID']
data = data.drop(columns=[col for col in columns_to_drop if col in data.columns], axis=1)


In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Drop irrelevant columns
columns_to_drop = ['TransactionID', 'PreviousTransactionDate', 'TransactionDate', 'IP Address', 'DeviceID']
data = data.drop(columns=columns_to_drop, axis=1)

# Encode the target variable
# Ensure that 'TransactionType' is properly mapped and handle missing values before conversion
data['TransactionType'] = data['TransactionType'].map({'Non-Fraud': 0, 'Fraud': 1})

# Check if any missing values in 'TransactionType'
if data['TransactionType'].isnull().any():
    print("Missing values found in 'TransactionType', filling with 0 (Non-Fraud).")
    data['TransactionType'].fillna(0, inplace=True)  # Fill missing values with 0 for non-fraud

# One-hot encode categorical columns
categorical_columns = ['Location', 'MerchantID', 'Channel', 'CustomerOccupation']
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# Fill missing values in features
data = data.fillna(data.mean())  # Fill missing numeric features with the mean

# Ensure 'Fraud' column is numeric and free of issues
y = data['TransactionType'].astype(int)

# Define features for logistic regression (use only numeric columns after encoding)
features = ['TransactionAmount', 'TransactionDuration', 'LoginAttempts', 'AccountBalance', 'CustomerAge']
features.extend([col for col in data.columns if col not in ['TransactionType']])  # Add one-hot encoded columns

# Prepare the dataset for modeling
X = data[features]

# Check if X contains any non-numeric data
if X.select_dtypes(include='object').shape[1] > 0:
    print("Non-numeric columns detected in features. Converting to numeric.")
    X = X.apply(pd.to_numeric, errors='coerce')  # Convert non-numeric values to NaN

# Handle any missing values that may appear after conversion
X.fillna(X.mean(), inplace=True)

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Train Logistic Regression model
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

# Predict fraud on the test set
y_pred = log_reg.predict(X_test)

# Evaluate model performance
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Non-Fraud', 'Fraud'], yticklabels=['Non-Fraud', 'Fraud'])
plt.title('Confusion Matrix', fontsize=16)
plt.xlabel('Predicted', fontsize=14)
plt.ylabel('Actual', fontsize=14)
plt.show()

# Add predictions to the dataset
data['LogReg_Fraud'] = log_reg.predict(X_scaled)

# Visualize fraud vs. non-fraud transactions
plt.figure(figsize=(12, 8))
sns.scatterplot(
    x=data['TransactionAmount'],
    y=data['AccountBalance'],
    hue=data['LogReg_Fraud'],
    palette={1: 'red', 0: 'blue'},
    alpha=0.7
)
plt.title('Logistic Regression Fraud Detection', fontsize=16)
plt.xlabel('Transaction Amount', fontsize=14)
plt.ylabel('Account Balance', fontsize=14)
plt.legend(title='Fraud', labels=['Non-Fraud', 'Fraud'], fontsize=12)
plt.grid(True)
plt.show()

# Save fraudulent transactions detected by Logistic Regression
log_reg_fraud_output_path = 'log_reg_fraud_transactions.csv'
log_reg_fraud_points = data[data['LogReg_Fraud'] == 1]
log_reg_fraud_points.to_csv(log_reg_fraud_output_path, index=False)

# Summary
total_log_reg_fraud_points = log_reg_fraud_points.shape[0]
print(f"Total Fraudulent Transactions Detected by Logistic Regression: {total_log_reg_fraud_points}")
print(f"Fraudulent transactions saved to: {log_reg_fraud_output_path}")
print(log_reg_fraud_points)


KeyError: "['TransactionID', 'PreviousTransactionDate', 'TransactionDate', 'IP Address', 'DeviceID'] not found in axis"

In [30]:
columns_to_drop = ['TransactionID', 'PreviousTransactionDate', 'TransactionDate', 'IP Address', 'DeviceID']
columns_to_drop = [col for col in columns_to_drop if col in data.columns]  # Drop only existing columns
data = data.drop(columns=columns_to_drop, axis=1)


In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Drop irrelevant columns
columns_to_drop = ['TransactionID', 'PreviousTransactionDate', 'TransactionDate', 'IP Address', 'DeviceID']
data = data.drop(columns=columns_to_drop, axis=1)

# Encode the target variable
# Ensure that 'TransactionType' is properly mapped and handle missing values before conversion
data['TransactionType'] = data['TransactionType'].map({'Non-Fraud': 0, 'Fraud': 1})

# Check if any missing values in 'TransactionType'
if data['TransactionType'].isnull().any():
    print("Missing values found in 'TransactionType', filling with 0 (Non-Fraud).")
    data['TransactionType'].fillna(0, inplace=True)  # Fill missing values with 0 for non-fraud

# One-hot encode categorical columns
categorical_columns = ['Location', 'MerchantID', 'Channel', 'CustomerOccupation']
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# Fill missing values in features
data = data.fillna(data.mean())  # Fill missing numeric features with the mean

# Ensure 'Fraud' column is numeric and free of issues
y = data['TransactionType'].astype(int)

# Define features for logistic regression (use only numeric columns after encoding)
features = ['TransactionAmount', 'TransactionDuration', 'LoginAttempts', 'AccountBalance', 'CustomerAge']
features.extend([col for col in data.columns if col not in ['TransactionType']])  # Add one-hot encoded columns

# Prepare the dataset for modeling
X = data[features]

# Check if X contains any non-numeric data
if X.select_dtypes(include='object').shape[1] > 0:
    print("Non-numeric columns detected in features. Converting to numeric.")
    X = X.apply(pd.to_numeric, errors='coerce')  # Convert non-numeric values to NaN

# Handle any missing values that may appear after conversion
X.fillna(X.mean(), inplace=True)

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Train Logistic Regression model
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

# Predict fraud on the test set
y_pred = log_reg.predict(X_test)

# Evaluate model performance
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Non-Fraud', 'Fraud'], yticklabels=['Non-Fraud', 'Fraud'])
plt.title('Confusion Matrix', fontsize=16)
plt.xlabel('Predicted', fontsize=14)
plt.ylabel('Actual', fontsize=14)
plt.show()

# Add predictions to the dataset
data['LogReg_Fraud'] = log_reg.predict(X_scaled)

# Visualize fraud vs. non-fraud transactions
plt.figure(figsize=(12, 8))
sns.scatterplot(
    x=data['TransactionAmount'],
    y=data['AccountBalance'],
    hue=data['LogReg_Fraud'],
    palette={1: 'red', 0: 'blue'},
    alpha=0.7
)
plt.title('Logistic Regression Fraud Detection', fontsize=16)
plt.xlabel('Transaction Amount', fontsize=14)
plt.ylabel('Account Balance', fontsize=14)
plt.legend(title='Fraud', labels=['Non-Fraud', 'Fraud'], fontsize=12)
plt.grid(True)
plt.show()

# Save fraudulent transactions detected by Logistic Regression
log_reg_fraud_output_path = 'log_reg_fraud_transactions.csv'
log_reg_fraud_points = data[data['LogReg_Fraud'] == 1]
log_reg_fraud_points.to_csv(log_reg_fraud_output_path, index=False)

# Summary
total_log_reg_fraud_points = log_reg_fraud_points.shape[0]
print(f"Total Fraudulent Transactions Detected by Logistic Regression: {total_log_reg_fraud_points}")
print(f"Fraudulent transactions saved to: {log_reg_fraud_output_path}")
print(log_reg_fraud_points)


KeyError: "['TransactionID', 'PreviousTransactionDate', 'TransactionDate', 'IP Address', 'DeviceID'] not found in axis"

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Prepare the dataset
features = ['TransactionAmount', 'TransactionDuration', 'LoginAttempts', 'AccountBalance', 'CustomerAge', 'TimeSinceLastTransaction']  # Update this as needed
target = 'TransactionType'

X = data[features]
y = data[target].map({'Non-Fraud': 0, 'Fraud': 1})  # Encode target variable

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Train Logistic Regression model
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

# Predict fraud on the test set
y_pred = log_reg.predict(X_test)

# Evaluate model performance
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Non-Fraud', 'Fraud'], yticklabels=['Non-Fraud', 'Fraud'])
plt.title('Confusion Matrix', fontsize=16)
plt.xlabel('Predicted', fontsize=14)
plt.ylabel('Actual', fontsize=14)
plt.show()

# Add predictions to the dataset
data['LogReg_Fraud'] = log_reg.predict(X_scaled)
data['Fraud'] |= data['LogReg_Fraud']

# Visualize fraud vs. non-fraud transactions
plt.figure(figsize=(12, 8))
sns.scatterplot(
    x=data['TransactionAmount'],
    y=data['AccountBalance'],
    hue=data['LogReg_Fraud'],
    palette={1: 'red', 0: 'blue'},
    alpha=0.7
)
plt.title('Logistic Regression Fraud Detection', fontsize=16)
plt.xlabel('Transaction Amount', fontsize=14)
plt.ylabel('Account Balance', fontsize=14)
plt.legend(title='Fraud', labels=['Non-Fraud', 'Fraud'], fontsize=12)
plt.grid(True)
plt.show()

# Save fraudulent transactions detected by Logistic Regression
log_reg_fraud_output_path = '/kaggle/working/log_reg_fraud_transactions.csv'
log_reg_fraud_points = data[data['LogReg_Fraud'] == 1]
log_reg_fraud_points.to_csv(log_reg_fraud_output_path, index=False)

# Summary
total_log_reg_fraud_points = log_reg_fraud_points.shape[0]
print(f"Total Fraudulent Transactions Detected by Logistic Regression: {total_log_reg_fraud_points}")
print(f"Fraudulent transactions saved to: {log_reg_fraud_output_path}")
print(log_reg_fraud_points)


ValueError: Input y contains NaN.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Prepare the dataset
features = ['TransactionAmount', 'TransactionDuration', 'LoginAttempts', 'AccountBalance', 'CustomerAge', 'TimeSinceLastTransaction']  # Update this as needed
target = 'TransactionType'

X = data[features]
y = data[target].map({'Non-Fraud': 0, 'Fraud': 1})  # Encode target variable

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Train Logistic Regression model
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

# Predict fraud on the test set
y_pred = log_reg.predict(X_test)

# Evaluate model performance
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Non-Fraud', 'Fraud'], yticklabels=['Non-Fraud', 'Fraud'])
plt.title('Confusion Matrix', fontsize=16)
plt.xlabel('Predicted', fontsize=14)
plt.ylabel('Actual', fontsize=14)
plt.show()

# Add predictions to the dataset
data['LogReg_Fraud'] = log_reg.predict(X_scaled)
data['Fraud'] |= data['LogReg_Fraud']

# Visualize fraud vs. non-fraud transactions
plt.figure(figsize=(12, 8))
sns.scatterplot(
    x=data['TransactionAmount'],
    y=data['AccountBalance'],
    hue=data['LogReg_Fraud'],
    palette={1: 'red', 0: 'blue'},
    alpha=0.7
)
plt.title('Logistic Regression Fraud Detection', fontsize=16)
plt.xlabel('Transaction Amount', fontsize=14)
plt.ylabel('Account Balance', fontsize=14)
plt.legend(title='Fraud', labels=['Non-Fraud', 'Fraud'], fontsize=12)
plt.grid(True)
plt.show()

# Save fraudulent transactions detected by Logistic Regression
log_reg_fraud_output_path = '/kaggle/working/log_reg_fraud_transactions.csv'
log_reg_fraud_points = data[data['LogReg_Fraud'] == 1]
log_reg_fraud_points.to_csv(log_reg_fraud_output_path, index=False)

# Summary
total_log_reg_fraud_points = log_reg_fraud_points.shape[0]
print(f"Total Fraudulent Transactions Detected by Logistic Regression: {total_log_reg_fraud_points}")
print(f"Fraudulent transactions saved to: {log_reg_fraud_output_path}")
print(log_reg_fraud_points)


ValueError: Input y contains NaN.

In [32]:
columns_to_drop = ['TransactionID', 'PreviousTransactionDate', 'TransactionDate', 'IP Address', 'DeviceID']
missing_cols = [col for col in columns_to_drop if col not in data.columns]
if missing_cols:
    print(f"Columns not found in DataFrame: {missing_cols}")
data = data.drop(columns=[col for col in columns_to_drop if col in data.columns], axis=1)


Columns not found in DataFrame: ['TransactionID', 'PreviousTransactionDate', 'TransactionDate', 'IP Address', 'DeviceID']


In [33]:
for col in data.columns:
    print(f"'{col}': {len(col)}")


'AccountID': 9
'TransactionAmount': 17
'TransactionType': 15
'CustomerAge': 11
'TransactionDuration': 19
'LoginAttempts': 13
'AccountBalance': 14
'TimeSinceLastTransaction': 24
'Location_Atlanta': 16
'Location_Austin': 15
'Location_Baltimore': 18
'Location_Boston': 15
'Location_Charlotte': 18
'Location_Chicago': 16
'Location_Colorado Springs': 25
'Location_Columbus': 17
'Location_Dallas': 15
'Location_Denver': 15
'Location_Detroit': 16
'Location_El Paso': 16
'Location_Fort Worth': 19
'Location_Fresno': 15
'Location_Houston': 16
'Location_Indianapolis': 21
'Location_Jacksonville': 21
'Location_Kansas City': 20
'Location_Las Vegas': 18
'Location_Los Angeles': 20
'Location_Louisville': 19
'Location_Memphis': 16
'Location_Mesa': 13
'Location_Miami': 14
'Location_Milwaukee': 18
'Location_Nashville': 18
'Location_New York': 17
'Location_Oklahoma City': 22
'Location_Omaha': 14
'Location_Philadelphia': 21
'Location_Phoenix': 16
'Location_Portland': 17
'Location_Raleigh': 16
'Location_Sacrament

In [34]:
columns_to_drop = ['TransactionID', 'PreviousTransactionDate', 'TransactionDate', 'IP Address', 'DeviceID']
# Check which columns exist in the DataFrame
columns_to_drop_existing = [col for col in columns_to_drop if col in data.columns]
# Drop only those that exist
data = data.drop(columns=columns_to_drop_existing, axis=1)
print(f"Dropped columns: {columns_to_drop_existing}")


Dropped columns: []


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Drop irrelevant columns
columns_to_drop = ['TransactionID', 'PreviousTransactionDate', 'TransactionDate', 'IP Address', 'DeviceID']
data = data.drop(columns=columns_to_drop, axis=1)

# Encode the target variable
# Ensure that 'TransactionType' is properly mapped and handle missing values before conversion
data['TransactionType'] = data['TransactionType'].map({'Non-Fraud': 0, 'Fraud': 1})

# Check if any missing values in 'TransactionType'
if data['TransactionType'].isnull().any():
    print("Missing values found in 'TransactionType', filling with 0 (Non-Fraud).")
    data['TransactionType'].fillna(0, inplace=True)  # Fill missing values with 0 for non-fraud

# One-hot encode categorical columns
categorical_columns = ['Location', 'MerchantID', 'Channel', 'CustomerOccupation']
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# Fill missing values in features
data = data.fillna(data.mean())  # Fill missing numeric features with the mean

# Ensure 'Fraud' column is numeric and free of issues
y = data['TransactionType'].astype(int)

# Define features for logistic regression (use only numeric columns after encoding)
features = ['TransactionAmount', 'TransactionDuration', 'LoginAttempts', 'AccountBalance', 'CustomerAge']
features.extend([col for col in data.columns if col not in ['TransactionType']])  # Add one-hot encoded columns

# Prepare the dataset for modeling
X = data[features]

# Check if X contains any non-numeric data
if X.select_dtypes(include='object').shape[1] > 0:
    print("Non-numeric columns detected in features. Converting to numeric.")
    X = X.apply(pd.to_numeric, errors='coerce')  # Convert non-numeric values to NaN

# Handle any missing values that may appear after conversion
X.fillna(X.mean(), inplace=True)

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Train Logistic Regression model
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

# Predict fraud on the test set
y_pred = log_reg.predict(X_test)

# Evaluate model performance
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Non-Fraud', 'Fraud'], yticklabels=['Non-Fraud', 'Fraud'])
plt.title('Confusion Matrix', fontsize=16)
plt.xlabel('Predicted', fontsize=14)
plt.ylabel('Actual', fontsize=14)
plt.show()

# Add predictions to the dataset
data['LogReg_Fraud'] = log_reg.predict(X_scaled)

# Visualize fraud vs. non-fraud transactions
plt.figure(figsize=(12, 8))
sns.scatterplot(
    x=data['TransactionAmount'],
    y=data['AccountBalance'],
    hue=data['LogReg_Fraud'],
    palette={1: 'red', 0: 'blue'},
    alpha=0.7
)
plt.title('Logistic Regression Fraud Detection', fontsize=16)
plt.xlabel('Transaction Amount', fontsize=14)
plt.ylabel('Account Balance', fontsize=14)
plt.legend(title='Fraud', labels=['Non-Fraud', 'Fraud'], fontsize=12)
plt.grid(True)
plt.show()

# Save fraudulent transactions detected by Logistic Regression
log_reg_fraud_output_path = 'log_reg_fraud_transactions.csv'
log_reg_fraud_points = data[data['LogReg_Fraud'] == 1]
log_reg_fraud_points.to_csv(log_reg_fraud_output_path, index=False)

# Summary
total_log_reg_fraud_points = log_reg_fraud_points.shape[0]
print(f"Total Fraudulent Transactions Detected by Logistic Regression: {total_log_reg_fraud_points}")
print(f"Fraudulent transactions saved to: {log_reg_fraud_output_path}")
print(log_reg_fraud_points)


KeyError: "None of [Index(['Location', 'MerchantID', 'Channel', 'CustomerOccupation'], dtype='object')] are in the [columns]"

In [38]:
import pandas as pd
import numpy as np
from random import choice

# Generate a dataset with 800 rows
np.random.seed(42)

# Data for professions
professions = ['Doctor', 'Lawyer', 'Engineer', 'Teacher', 'Artist', 'Nurse']

# Generate random data for each column
data = {
    'Profession': [choice(professions) for _ in range(800)],
    'Income': np.random.randint(30000, 150000, 800),
    'Credit_card_number': [''.join([str(np.random.randint(0, 10)) for _ in range(16)]) for _ in range(800)],
    'Expiry': [f"{np.random.randint(1, 13):02}/{np.random.randint(21, 30)}" for _ in range(800)],
    'Security_Code': [np.random.randint(100, 999) for _ in range(800)],
    'Fraud': np.random.choice([0, 1], 800, p=[0.85, 0.15])  # Imbalanced classes, 15% fraud
}

# Create a DataFrame
df = pd.DataFrame(data)

# Introduce some missing values randomly
for column in ['Income', 'Credit_card_number', 'Expiry', 'Security_Code']:
    missing_indices = np.random.choice(df.index, size=int(0.1 * len(df)), replace=False)
    df.loc[missing_indices, column] = np.nan

# Show the first few rows of the dataset
df.head()


,Profession,Income,Credit_card_number,Expiry,Security_Code,Fraud
0,Nurse,45795.0,9144527053068335,01/25,410.0,0
1,Nurse,30860.0,NaN,02/29,373.0,0
2,Artist,133694.0,8087054594544322,07/28,134.0,0
3,Lawyer,NaN,3818004552689757,06/29,316.0,0
4,Lawyer,140268.0,4793979148350804,06/27,820.0,0


In [40]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('fraud-detection-dataset.csv')

# Check missing values
print("Missing values before handling:")
print(data.isnull().sum())

# Handle missing values
# Fill missing categorical column 'Profession' with the mode
data['Profession'] = data['Profession'].fillna(data['Profession'].mode()[0])

# Fill missing numerical columns with the mean
data['Income'] = data['Income'].fillna(data['Income'].mean())
data['Security_Code'] = data['Security_Code'].fillna(data['Security_Code'].mean())

# Fill missing target column 'Fraud' with the mode
data['Fraud'] = data['Fraud'].fillna(data['Fraud'].mode()[0])

# Fill missing 'Expiry' with the mode
data['Expiry'] = data['Expiry'].fillna(data['Expiry'].mode()[0])

# Feature Engineering: Create 'Months_to_expiry' column
current_date = datetime.now()

def calculate_months_to_expiry(expiry):
    exp_month, exp_year = map(int, expiry.split("/"))
    expiry_date = datetime(year=2000 + exp_year, month=exp_month, day=1)
    return max(0, (expiry_date.year - current_date.year) * 12 + (expiry_date.month - current_date.month))

data['Months_to_expiry'] = data['Expiry'].apply(calculate_months_to_expiry)

# One-hot encode the 'Profession' column
data = pd.get_dummies(data, columns=['Profession'], drop_first=True)

# Drop unnecessary columns
data = data.drop(columns=['Credit_card_number', 'Expiry'])

# Recheck for any remaining missing values
print("Missing values after handling:")
print(data.isnull().sum())

# Split the data into features (X) and target (y)
X = data.drop(columns=['Fraud'])
y = data['Fraud']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Train a Random Forest model using Gini index
model = RandomForestClassifier(criterion='gini', random_state=42)
model.fit(X_train, y_train)

# Predictions and evaluation
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Feature importance visualization
feature_importances = pd.Series(model.feature_importances_, index=X.columns)
feature_importances.sort_values(ascending=False).plot(kind='bar', title='Feature Importance', figsize=(10, 6))
plt.show()


ImportError: cannot import name '_check_n_features' from 'sklearn.utils.validation' (c:\Users\LENOVO\anaconda3\Lib\site-packages\sklearn\utils\validation.py)